In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

#pip install bs4
#pip install selenium
#pip install webdriver_manager
#pip install blinker==1.6.2

In [251]:
SCROLL_PAUSE_TIME = 3
pages = int(42)
cards = []

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
service = Service(ChromeDriverManager().install())

for i in range(1, pages+1):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    url = "https://www.zapimoveis.com.br/venda/apartamentos/go+goiania++setor-marista/?pagina=" + str(i)
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        try:
            img_element = driver.find_element(By.XPATH, "//img[contains(@class, 'campaign__container-image')]")
            driver.execute_script("arguments[0].scrollIntoView();", img_element)
        except:
            pass
        
        time.sleep(SCROLL_PAUSE_TIME)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    anuncios = soup.find('div', class_="listing-wrapper__content").find_all('div', class_="BaseCard_card__content__pL2Vc w-full p-3")

    for anuncio in anuncios:
        card = {}
        lista_valores = anuncio.find_all('p')
        valor_limpo = [valor.text.strip() for valor in lista_valores]
        
        card['Endereço'] = valor_limpo[0]
        card['Preço'] = valor_limpo[1].replace("A partir de ", "")

        lista_carac = anuncio.find_all('li')
        carac_limpo = [carac.text.strip() for carac in lista_carac]

        card['Área (m2)'] = carac_limpo[0].replace(" m²", "")
        card['Quartos'] = carac_limpo[1]
        card['Banheiros'] = carac_limpo[2]
        card['Garagem'] = carac_limpo[3] if len(carac_limpo) > 3 else 0

        cards.append(card)

    driver.quit()


In [2]:
#pip install pandas
import pandas as pd

df = pd.DataFrame(cards)
df

NameError: name 'cards' is not defined

In [287]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4337 entries, 0 to 4336
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Endereço   4337 non-null   object
 1   Preço      4337 non-null   object
 2   Área (m2)  4337 non-null   object
 3   Quartos    4337 non-null   object
 4   Banheiros  4337 non-null   object
 5   Garagem    4337 non-null   object
dtypes: object(6)
memory usage: 203.4+ KB


None

In [288]:
df.value_counts()

Endereço  Preço         Área (m2)  Quartos  Banheiros  Garagem
Rua 138   R$ 1.750.000  214        4        5          3          23
Rua 1124  R$ 1.500.000  125        3        4          2          16
Rua 143   R$ 930.000    115        3        7          2          15
Rua 147   R$ 1.370.000  115        3        7          2          15
Rua 1138  R$ 1.500.000  121        3        4          2          14
                                                                  ..
Rua T 53  R$ 7.068.256  393        4        6          4           1
          R$ 7.490.000  420        5        7          6           1
          R$ 710.000    60         2        3          2           1
          R$ 1.050.000  106        3        2          2           1
                        104        3        4          2           1
Name: count, Length: 2540, dtype: int64

In [289]:
df = df.drop_duplicates()
df

,Endereço,Preço,Área (m2),Quartos,Banheiros,Garagem
0,Rua Amélia Artiaga Jardim,R$ 1.250.000,116,3,4,2
1,Rua 1141,R$ 4.190.000,354,4,5,3
2,Rua 1128,R$ 1.650.000,166,3,4,3
3,,R$ 1.395.000,138,3,4,2
4,Rua 145,R$ 1.060.000,130,3,5,3
...,...,...,...,...,...,...
4332,,R$ 1.260.000,137,3,1,2
4333,,R$ 1.420.000,130,3,3,2
4334,,R$ 4.689.000,317,4,6,4
4335,,R$ 880.000,99,3,1,2


In [1]:
#df = df[df['Endereço'].str.strip() != ""]
df.to_csv("ApartamentosMarista.csv", index=False, sep=";", encoding="latin1")

NameError: name 'df' is not defined